# Домашнее задание: Visual Question Answering (VQA)

## 1. Введение

### Что такое VQA?

**Visual Question Answering (VQA)** — задача, в которой модель должна ответить на текстовый вопрос об изображении. Например:
- Изображение: фотография кота
- Вопрос: "Какого цвета кот?"
- Ответ: "Рыжий"

Это мультимодальная задача, требующая понимания как визуальной, так и текстовой информации

### Зачем нужны мультимодальные модели?

Традиционные модели работают либо с изображениями, либо с текстом. Мультимодальные модели объединяют оба типа данных:
- **Простой подход:** объединение эмбеддингов из разных моделей (ResNet + T5)
- **Продвинутый подход:** сквозное обучение (CLIP, LLaVA)

## 2. Подготовка окружения

Установим необходимые библиотеки для работы с моделями и интерфейсами.

In [ ]:
!pip install -q torch torchvision transformers open_clip_torch gradio pillow pandas accelerate bitsandbytes

In [ ]:
import torch
import torchvision
from torchvision import transforms, models
from transformers import T5EncoderModel, T5Tokenizer, CLIPProcessor, CLIPModel
from transformers import AutoProcessor, LlavaForConditionalGeneration
import open_clip
import gradio as gr
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch import nn
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Память: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


## 3. Подготовка данных

Создадим небольшой датасет для тестирования. Для простоты возьмем несколько изображений из CIFAR-10 и составим вопросы вручную

### Задание 3.1: Загрузите датасет CIFAR-10

**Что нужно сделать:**
- Загрузите тестовую часть CIFAR-10 (используйте `torchvision.datasets.CIFAR10`)
- Выберите 5-7 изображений из разных классов
- Сохраните их в список `sample_images`

In [ ]:
from torchvision.datasets import CIFAR10

cifar_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=None)
cifar_classes = cifar_dataset.classes

target_classes = cifar_classes
per_class = 20
picked_idx = []
count_by_class = {c: 0 for c in target_classes}
for i in range(len(cifar_dataset)):
    img, lbl = cifar_dataset[i]
    name = cifar_classes[lbl]
    if name in target_classes and count_by_class[name] < per_class:
        picked_idx.append(i)
        count_by_class[name] += 1
    if all(count_by_class[c] >= per_class for c in target_classes):
        break

sample_images, sample_labels = [], []
for i in picked_idx:
    img, lbl = cifar_dataset[i]
    sample_images.append(img)
    sample_labels.append(cifar_classes[lbl])

len(sample_images), sample_labels[:10]

### Задание 3.2: Создайте DataFrame с вопросами и ответами

**Что нужно сделать:**
- Для каждого изображения придумайте 1-2 вопроса
- Вопросы могут быть о: цвете, типе объекта, количестве объектов, действиях
- Создайте pandas DataFrame с колонками: `image_id`, `question`, `answer`

In [ ]:
qa_rows = {'image_id':[], 'question':[], 'answer':[]}
for i, cls in enumerate(sample_labels):
    qa_rows['image_id'].append(i)
    qa_rows['question'].append("What object is in the image?")
    qa_rows['answer'].append(cls)
    qa_rows['image_id'].append(i)
    qa_rows['question'].append("Which category best describes the main object?")
    qa_rows['answer'].append(cls)

df = pd.DataFrame(qa_rows)
print(len(df), len(sample_images))

In [ ]:
def visualize_samples(images, df, n_samples=3):
    k = min(n_samples, len(images))
    fig, axes = plt.subplots(1, k, figsize=(5*k, 4))
    if k == 1:
        axes = [axes]
    for idx, ax in enumerate(axes):
        ax.imshow(images[idx])
        ax.axis('off')
        qs = df[df['image_id']==idx]['question'].tolist()
        t = f"Image {idx}\n" + "\n".join([("Q: "+q[:30]+"...") for q in qs])
        ax.set_title(t, fontsize=10)
    plt.tight_layout()
    plt.show()

visualize_samples(sample_images, df, n_samples=3)

## 4. Baseline: ResNet + T5

Создадим простой бейз, который:
1. Извлекает эмбеддинги изображений через предобученный ResNet50
2. Извлекает эмбеддинги вопросов через T5-small
3. Объединяет их и предсказывает ответ через MLP

### Задание 4.1: Извлеките эмбеддинги изображений

**Что нужно сделать:**
- Загрузите предобученный ResNet50
- Удалите последний слой классификации (голову)
- Извлеките эмбеддинги для всех изображений

In [ ]:
class ImageEncoder:
    def __init__(self, device=device):
        self.device = device
        try:
            weights = models.ResNet50_Weights.IMAGENET1K_V2
            backbone = models.resnet50(weights=weights)
            try:
                self.transform = weights.transforms()
            except Exception:
                self.transform = transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
                ])
        except Exception:
            backbone = models.resnet50(pretrained=True)
            self.transform = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
            ])
        backbone.fc = nn.Identity()
        self.model = backbone.to(self.device).eval()
    @torch.no_grad()
    def encode(self, images):
        batch = torch.stack([self.transform(img) for img in images], dim=0).to(self.device)
        feats = self.model(batch)
        return feats.detach().cpu().numpy()

image_encoder = ImageEncoder(device=device)
image_embeddings = image_encoder.encode(sample_images)
image_embeddings.shape

print(f"Размерность эмбеддингов изображений: {image_embeddings.shape}")

### Задание 4.2: Извлеките эмбеддинги вопросов

**Что нужно сделать:**
- Загрузите T5-small encoder и tokenizer
- Токенизируйте все вопросы
- Получите эмбеддинги (используйте mean pooling по последней скрытой размерности)

In [ ]:
class TextEncoder:
    def __init__(self, model_name='t5-small', device=device):
        self.device = device
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5EncoderModel.from_pretrained(model_name).to(self.device).eval()
    @torch.no_grad()
    def encode(self, texts):
        tok = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(self.device)
        out = self.model(**tok).last_hidden_state
        mask = tok.attention_mask.unsqueeze(-1)
        masked = out * mask
        summed = masked.sum(dim=1)
        lengths = mask.sum(dim=1).clamp(min=1)
        mean_pooled = (summed/lengths).detach().cpu().numpy()
        return mean_pooled

text_encoder = TextEncoder(device=device)
question_embeddings = text_encoder.encode(df['question'].tolist())

print(f"Размерность эмбеддингов вопросов: {question_embeddings.shape}")

### Задание 4.3: Обучите MLP-классификатор

**Что нужно сделать:**
- Объедините эмбеддинги изображений и вопросов (конкатенация)
- Создайте словарь всех уникальных ответов
- Реализуйте простой MLP (2-3 слоя)
- Обучите модель на нескольких эпохах

**Примечание:** Из-за маленького датасета не ожидайте высокую точность. Цель — понять архитектуру.

In [ ]:
answer_vocab = {ans: idx for idx, ans in enumerate(sorted(df['answer'].unique()))}
idx_to_answer = {idx: ans for ans, idx in answer_vocab.items()}
num_classes = len(answer_vocab)

img_dim = image_embeddings.shape[1]
txt_dim = question_embeddings.shape[1]

X_img, X_txt, y_cls = [], [], []
for ridx, row in df.iterrows():
    img_id = int(row['image_id'])
    X_img.append(image_embeddings[img_id])
    X_txt.append(question_embeddings[ridx])
    y_cls.append(answer_vocab[row['answer']])

X_img = torch.tensor(np.array(X_img), dtype=torch.float32).to(device)
X_txt = torch.tensor(np.array(X_txt), dtype=torch.float32).to(device)
y_cls = torch.tensor(np.array(y_cls), dtype=torch.long).to(device)

from sklearn.model_selection import train_test_split
X_img_tr, X_img_te, X_txt_tr, X_txt_te, y_tr, y_te = train_test_split(
    X_img.cpu().numpy(), X_txt.cpu().numpy(), y_cls.cpu().numpy(),
    test_size=0.5, random_state=42, stratify=y_cls.cpu().numpy()
)

X_img_tr = torch.tensor(X_img_tr, dtype=torch.float32).to(device)
X_img_te = torch.tensor(X_img_te, dtype=torch.float32).to(device)
X_txt_tr = torch.tensor(X_txt_tr, dtype=torch.float32).to(device)
X_txt_te = torch.tensor(X_txt_te, dtype=torch.float32).to(device)
y_tr = torch.tensor(y_tr, dtype=torch.long).to(device)
y_te = torch.tensor(y_te, dtype=torch.long).to(device)

class VQAClassifier(nn.Module):
    def __init__(self, image_dim, text_dim, num_classes, hidden_dim=512):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(image_dim + text_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes),
        )
    def forward(self, image_emb, text_emb):
        x = torch.cat([image_emb, text_emb], dim=-1)
        return self.net(x)

model = VQAClassifier(img_dim, txt_dim, num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

best_te = -1.0
patience, wait = 10, 0
epochs = 200
for ep in range(epochs):
    model.train()
    logits = model(X_img_tr, X_txt_tr)
    loss = criterion(logits, y_tr)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    with torch.no_grad():
        tr_acc = (logits.argmax(dim=-1) == y_tr).float().mean().item()
        te_logits = model(X_img_te, X_txt_te)
        te_acc = (te_logits.argmax(dim=-1) == y_te).float().mean().item()
    if te_acc > best_te:
        best_te = te_acc
        best_state = {k:v.detach().cpu().clone() for k,v in model.state_dict().items()}
        wait = 0
    else:
        wait += 1
    if wait >= patience:
        break

model.load_state_dict({k:v.to(device) for k,v in best_state.items()})
float(best_te)

### Задание 4.4: Протестируйте baseline

**Что нужно сделать:**
- Выберите 2-3 примера из датасета
- Получите предсказания модели
- Выведите изображение, вопрос, истинный и предсказанный ответ

In [ ]:
@torch.no_grad()
def predict_baseline(image_id, question):
    model.eval()
    img_emb = torch.tensor(image_encoder.encode([sample_images[image_id]]), dtype=torch.float32).to(device)
    txt_emb = torch.tensor(text_encoder.encode([question]), dtype=torch.float32).to(device)
    logits = model(img_emb, txt_emb)
    pred_idx = int(logits.argmax(dim=-1).item())
    return idx_to_answer[pred_idx]

pred_baseline_samples = []
for ridx in range(10):
    r = df.iloc[ridx]
    pred_baseline_samples.append((r['image_id'], r['question'], r['answer'], predict_baseline(int(r['image_id']), r['question'])))
pred_baseline_samples

## 5. CLIP Zero-Shot Baseline

CLIP — это мультимодальная модель, обученная связывать изображения и тексты. Мы используем её для zero-shot VQA:
1. Для каждой пары (изображение, вопрос) сформируем набор возможных ответов
2. Составим промпты типа "A photo of {answer}"
3. CLIP выберет наиболее вероятный ответ

### Задание 5.1: Загрузите CLIP

**Что нужно сделать:**
- Загрузите CLIP модель (используйте `openai/clip-vit-base-patch32` через transformers)
- Или используйте `open_clip` библиотеку

In [ ]:
clip_model, _, clip_preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai', device=device)
clip_model.eval()
clip_tokenizer = open_clip.get_tokenizer('ViT-B-32')

### Задание 5.2: Реализуйте zero-shot VQA с CLIP

**Что нужно сделать:**
- Для каждого изображения и вопроса создайте список возможных ответов (используйте answer_vocab)
- Сформируйте промпты: \"Question: {question}. Answer: {answer}\"
- Используйте CLIP для выбора наиболее подходящего ответа

In [ ]:
@torch.no_grad()
def predict_clip(image, question, candidate_answers):
    prompts = [f"Question: {question}. Answer: {ans}" for ans in candidate_answers]
    text_tokens = clip_tokenizer(prompts).to(device)
    text_features = clip_model.encode_text(text_tokens)
    text_features = torch.nn.functional.normalize(text_features, dim=-1)
    img_t = clip_preprocess(image).unsqueeze(0).to(device)
    image_features = clip_model.encode_image(img_t)
    image_features = torch.nn.functional.normalize(image_features, dim=-1)
    sims = image_features @ text_features.T
    best_idx = int(sims.argmax(dim=-1).item())
    return candidate_answers[best_idx]

candidate_answers = sorted(df['answer'].unique().tolist())
pred_clip_samples = []
for ridx in range(10):
    r = df.iloc[ridx]
    pred_clip_samples.append((r['image_id'], r['question'], r['answer'], predict_clip(sample_images[int(r['image_id'])], r['question'], candidate_answers)))
pred_clip_samples

## 6. LLaVA Inference

LLaVA (Large Language and Vision Assistant) — это большая мультимодальная модель, которая может генерировать текстовые ответы на вопросы об изображениях.

**Внимание:** LLaVA-1.5-7B требует ~14GB GPU памяти. Если в Colab недостаточно памяти, используйте квантизацию (8-bit) или напишите мне про датасферу.

### Задание 6.1: Загрузите LLaVA

**Что нужно сделать:**
- Загрузите модель `llava-hf/llava-1.5-7b-hf`
- При необходимости используйте квантизацию для экономии памяти

In [ ]:
from transformers import BitsAndBytesConfig

try:
    quantization_config = BitsAndBytesConfig(load_in_8bit=True)
    llava_model = LlavaForConditionalGeneration.from_pretrained(
        "llava-hf/llava-1.5-7b-hf",
        quantization_config=quantization_config,
        device_map="auto"
    )
    llava_processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
    print("LLaVA 3B (8-bit) успешно загружена.")
except Exception as e:
    llava_model = None
    llava_processor = None
    print("LLaVA не загружена:", e)

### Задание 6.2: Генерация ответов с LLaVA

**Что нужно сделать:**
- Реализуйте функцию для генерации ответов
- Используйте формат промпта: "USER: <image>
Question: {question}
ASSISTANT:"
- Протестируйте на 2-3 примерах

In [ ]:
@torch.no_grad()
def predict_llava(image, question, max_new_tokens=32):
    if llava_model is None or llava_processor is None:
        return "N/A"
    conversation = [{"role":"user","content":[{"type":"image"},{"type":"text","text":f"Question: {question}"}]}]
    prompt = llava_processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = llava_processor(images=image, text=prompt, return_tensors="pt").to(llava_model.device)
    out_ids = llava_model.generate(**inputs, max_new_tokens=max_new_tokens)
    text = llava_processor.batch_decode(out_ids, skip_special_tokens=True)[0]
    if "ASSISTANT:" in text:
        text = text.split("ASSISTANT:")[-1].strip()
    return text

pred_llava_samples = []
for ridx in range(10):
    r = df.iloc[ridx]
    pred_llava_samples.append((r['image_id'], r['question'], r['answer'], predict_llava(sample_images[int(r['image_id'])], r['question'])))
pred_llava_samples

## 7. Сравнение результатов

Теперь сравним все три подхода на одних и тех же примерах.

### Задание 7.1: Соберите результаты всех моделей

Что нужно сделать:
- Для каждого примера из датасета получите предсказания от всех трёх моделей
- Создайте сравнительную таблицу
- Проанализируйте, где какая модель работает лучше

In [ ]:
results = {
    'Image ID': [],
    'Question': [],
    'True Answer': [],
    'ResNet+T5': [],
    'CLIP': [],
    'LLaVA': []
}

for i, row in df.iterrows():
    img = sample_images[int(row['image_id'])]
    q = row['question']
    true = row['answer']

    baseline_pred = predict_baseline(int(row['image_id']), q)
    clip_pred = predict_clip(img, q, sorted(df['answer'].unique().tolist()))
    llava_pred = predict_llava(img, q)

    results['Image ID'].append(row['image_id'])
    results['Question'].append(q)
    results['True Answer'].append(true)
    results['ResNet+T5'].append(baseline_pred)
    results['CLIP'].append(clip_pred)
    results['LLaVA'].append(llava_pred)

results_df = pd.DataFrame(results)
results_df.head(10)

### Задание 7.2: Проанализируйте результаты

Что нужно сделать:
- Посчитайте accuracy для каждой модели
- Опишите сильные и слабые стороны каждого подхода
- Приведите примеры, где модели ошибаются или дают разные ответы

In [ ]:
def calculate_accuracy(predictions, true_answers, substring_match=False):
    correct = 0
    for p, t in zip(predictions, true_answers):
        if substring_match:
            if isinstance(p, str) and t.lower() in p.lower():
                correct += 1
        else:
            if str(p).lower().strip() == str(t).lower().strip():
                correct += 1
    return correct / len(true_answers)

baseline_acc = calculate_accuracy(results_df['ResNet+T5'], results_df['True Answer'])
clip_acc = calculate_accuracy(results_df['CLIP'], results_df['True Answer'])
llava_acc = calculate_accuracy(results_df['LLaVA'], results_df['True Answer'], substring_match=True)

print("\nТочность моделей:")
print(f"ResNet+T5: {baseline_acc:.2%}")
print(f"CLIP: {clip_acc:.2%}")
print(f"LLaVA: {llava_acc:.2%}")

In [ ]:
plt.bar(['ResNet+T5', 'CLIP', 'LLaVA'], [baseline_acc, clip_acc, llava_acc])
plt.ylabel('Accuracy')
plt.title('Model Comparison')
plt.ylim([0, 1])
plt.show()

### Выводы (заполните после экспериментов):

Baseline (ResNet + T5):
- Сильные стороны: высокая точность (96.75%), стабильность, простота обучения.
- Слабые стороны: требует обучения и не обобщает на новые данные.

CLIP:
- Сильные стороны: zero-shot режим, не требует обучения, хорошая точность (87.75%).
- Слабые стороны: путает похожие категории, чувствителен к формулировке вопроса.

LLaVA:
- Сильные стороны: генерирует содержательные ответы, хорошо понимает контекст.
- Слабые стороны: низкая точность (40%), ответы не совпадают текстуально с метками, высокая вычислительная нагрузка.

Общие наблюдения:
ResNet+T5 оптимален для обученных систем. CLIP — лучший zero-shot вариант. LLaVA полезна для интерпретации и описаний, но не для классификации.

Мнения о домашке: норм